In [9]:
import glob
import networkx as nx
import matplotlib.pyplot as plt
import requests
import base64

# %matplotlib widget

In [10]:
FILE_COUNT = 6

### Merkle Tree

In [11]:
class MerkleTree:
    def __init__(self, value):
        self.left = None
        self.right = None
        self.value = value
        self.hash_value = TTH(value)

    def print_tree(self):
        if self.left:
            self.left.print_tree()
        if self.right:
            self.right.print_tree()
        print(f"{self.hash_value} ", end='')

### Toy Tetragraph Hash

In [12]:
# returns the position of an alphabetic char in sequence 0 ~ 25
def indexOf(ch):
    if ch == '0':  # this part only used by TTH
        return 0
    return ord(ch)-ord('a')

def compression(block, total):
    current_total = total

    # ROUND 1
    for i in range(4):
        sum = 0
        for j in range(i, 16, 4):
            sum += indexOf(block[j])
        current_total[i] = (current_total[i] + sum) % 26

    # ROUND 2
    matrix = block[1:4] + list(block[0]) + block[6:8] + block[4:6] + list(block[-5]) \
        + block[8:11] + list(reversed(block[12:16]))

    for i in range(4):
        sum = 0
        for j in range(i, 16, 4):
            sum += indexOf(matrix[j])
        current_total[i] = (current_total[i] + sum) % 26

    return current_total

def TTH(msg):
    # remove non-alphas
    alpha_msg = ""
    for ch in msg:
        if ch.isalpha():
            alpha_msg += ch.lower()

    # divide message into blocks of 16
    blocks = [alpha_msg[i:i+16] for i in range(0, len(alpha_msg), 16)]

    # pad with zeros
    if len(blocks[-1]) < 16:
        blocks[-1] = blocks[-1] + "0" * (16 - len(blocks[-1]))

    total = [0, 0, 0, 0]
    for b in blocks:
        total = compression(list(b), total)

    hash = ""
    for el in total:
        hash += chr(el+ord('A'))

    return hash

### Build Merkle Tree

In [13]:
# location: local or server
# G: networkx object
def build_tree(location, G=None):
    nodes, files = [], []
    if location == 'l':
        files = sorted(glob.glob('tests/local/*.txt'))
        for file in files:
            with open(file) as f:
                fc = f.readlines()
            nodes.append(MerkleTree("".join(fc)))
    else:
        # will use github
        for i in range(1, FILE_COUNT+1):
            url = 'https://api.github.com/repos/chiefdondjeu/472Proj3ServerSim/contents/file{i}.txt'.format(i=i)
            req = requests.get(url)
            if req.status_code == requests.codes.ok:
                fc = base64.b64decode(req.json()['content']).decode('utf-8')
                nodes.append(MerkleTree(fc))
            else:
                print('file{i}.txt not found'.format(i=i))

    # build actual tree
    if G is None:
        while len(nodes) != 1:
            parents = []
            for i in range(0, len(nodes), 2):
                child1 = nodes[i]
                if i+1 < len(nodes):
                    child2 = nodes[i+1]
                else:
                    parents.append(nodes[i])
                    break

                combined_hash = child1.hash_value + child2.hash_value
                parent = MerkleTree(combined_hash)
                parent.left = child1
                parent.right = child2
                parents.append(parent)
            nodes = parents
        return nodes[0]
    else:
        while len(nodes) != 1:
            parents = []
            for i in range(0, len(nodes), 2):
                child1 = nodes[i]
                if i+1 < len(nodes):
                    child2 = nodes[i+1]
                else:
                    parents.append(nodes[i])
                    break

                combined_hash = child1.hash_value + child2.hash_value
                parent = MerkleTree(combined_hash)
                parent.left = child1
                parent.right = child2
                parents.append(parent)

                G.add_node(child1.hash_value)
                G.add_node(child2.hash_value)
                G.add_node(parent.hash_value)
                G.add_edge(parent.hash_value, child1.hash_value)
                G.add_edge(parent.hash_value, child2.hash_value)
            nodes = parents
        return nodes[0]

### Post Order

In [14]:
tree = build_tree('l')
tree.print_tree()

KDDG DCZO PWLG IQXB HPRX WZWV DBJP VGAZ DVUO YPWD YCSK 

### Fancy Output

In [15]:
def add_file_nodes(G):
	count = 1
	for i in range(0, FILE_COUNT+1, 3):
		G.add_edge(list(G.nodes)[i], 'file{i}'.format(i=count))
		count += 1
		G.add_edge(list(G.nodes)[i+1], 'file{i}'.format(i=count))
		count += 1

In [16]:
plt.subplots(figsize=(15, 5))

G = nx.DiGraph()
build_tree('l', G)
add_file_nodes(G)
# fig = plt.figure()
plt.subplot(1, 2, 1)
pos = nx.nx_pydot.graphviz_layout(G, prog="dot")
nx.draw(G, pos, with_labels=True, node_size=1500, arrows=True, arrowsize=22)

Gs = nx.DiGraph()
build_tree('s', Gs)
add_file_nodes(Gs)
# fig = plt.figure(2)
plt.subplot(1, 2, 2)
pos = nx.nx_pydot.graphviz_layout(Gs, prog="dot")
nx.draw(Gs, pos, with_labels=True, node_size=1500, arrows=True, arrowsize=22)

plt.tight_layout()

file1.txt
file2.txt
file3.txt
file4.txt
file5.txt
file6.txt


In [ ]:
Gs = nx.DiGraph()
build_tree('s', Gs)
add_file_nodes(Gs)
fig = plt.figure(2)
pos = nx.nx_pydot.graphviz_layout(Gs, prog="dot")
nx.draw(Gs, pos, with_labels=True, node_size=1500, arrows=True, arrowsize=22)